In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
import sklearn
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

In [32]:
df = pd.read_csv('covid_toy.csv')
df.head(5)

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [6]:
df['fever'].isnull().sum()

10

In [7]:
df['gender'].value_counts()

gender
Female    59
Male      41
Name: count, dtype: int64

In [8]:
df['cough'].value_counts()

cough
Mild      62
Strong    38
Name: count, dtype: int64

In [9]:
df['city'].value_counts()

city
Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: count, dtype: int64

In [11]:
from sklearn.model_selection import train_test_split

In [13]:
X_train , X_test , y_train , y_test = train_test_split(df.drop(columns = ['has_covid']) , df['has_covid'] , test_size = 0.2)

## Simple Preprocessing

In [21]:
# adding simpleimputer to fever column
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])
X_test_fever = si.fit_transform(X_test[['fever']])

X_train_fever.shape

(80, 1)

In [22]:
# ordinal Encoding on cough column

oe = OrdinalEncoder(categories = [['Mild' , 'Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])
X_test_cough = oe.fit_transform(X_test[['cough']])

X_test_cough.shape

(20, 1)

In [25]:
# OnehotEncoding to gender and city column

ohe = OneHotEncoder(drop = 'first' , sparse = False)
X_train_gender_city = ohe.fit_transform(X_train[['gender' , 'city']])
X_test_gender_city = ohe.fit_transform(X_test[['gender' , 'city']])

X_train_gender_city.shape

C:\Users\Parveen\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\Parveen\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(80, 4)

In [27]:
# extract age from original X_train data
X_train_age =  X_train.drop(columns = ['gender' , 'fever' , 'cough' , 'city']).values
X_test_age = X_test.drop(columns = ['gender' , 'fever' , 'cough' , 'city']).values

In [29]:
# Concatenation of all the columns

X_train_transformed = np.concatenate((X_train_age , X_train_fever , X_train_cough , X_train_gender_city), axis = 1)
X_test_transformed = np.concatenate((X_test_age , X_test_fever , X_test_cough , X_test_gender_city ), axis = 1 )

In [34]:
X_train_transformed.shape


(80, 7)

## Using Column Transformer (Scikit Learn)

In [35]:
from sklearn.compose import ColumnTransformer

In [38]:
transformer = ColumnTransformer(transformers = [
              ('tnf1' , SimpleImputer() , ["fever"]),
              ('tnf2', OrdinalEncoder(categories = [['Mild' , 'Strong']]), ['cough']),
              ('tnf3' , OneHotEncoder(drop = 'first' , sparse = False) , ['gender' , 'city'])
    
] , remainder = 'passthrough')

In [40]:
transformer.fit_transform(X_train).shape

C:\Users\Parveen\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(80, 7)

In [41]:
transformer.fit_transform(X_test).shape

C:\Users\Parveen\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(20, 7)